In [31]:
import json
import requests

sql_jsvee_n_sqlknot = json.load(open("./old-catalog.json", "r"))
sql_jsvee_n_sqlknot = [each 
                       for each in sql_jsvee_n_sqlknot 
                       if each["domain_id"] == "sql" and 
                          each["provider_id"] in ["JSVEE", "sqlknot"]]
len(sql_jsvee_n_sqlknot)

274

In [ ]:

affiliations = {
    "Ayca Cebi": "",
    "Administrator": "",
    "Kamil Akhuseyinoglu": "University of Pittsburgh",
    "Rully Hendrawan": "University of Pittsburgh",
}

uses_by = {
    "Administrator": "Administrator",
    "dguerra": "Daniel Guerra",
    "Jordan Barria, Jordan Barria": "Jordan Barria",
    "Kamil Akhuseyinoglu": "Kamil Akhuseyinoglu",
    "Rafael Araujo": "Rafael Araujo",
    "Roya Hosseini": "Roya Hosseini",
    "Rully Agus Hendrawan, Rully Agus Hendrawan": "Rully Agus Hendrawan",
}

def get_used_in(item):
    return requests.get(f"http://adapt2.sis.pitt.edu/next.course-authoring/api/catalog/contents/{item['id']}/courses").json()

def get_agg_kcs(item):
    return requests.get(f"http://adapt2.sis.pitt.edu/next.course-authoring/api/catalog/contents/aggregate-concepts?contentId={item['id']}").json()

def get_um2_kcs(item):
    return requests.get(f"http://adapt2.sis.pitt.edu/next.course-authoring/api/catalog/contents/um2-concepts?activityName={item['short_name']}").json()

api_token = open('api_token.txt').read().strip()
for index, item in enumerate(sql_jsvee_n_sqlknot):
    url = item['url']
    print(f"processing {index+1}/{len(sql_jsvee_n_sqlknot)}: {url}")
    
    is_dbqa = 'http://adapt2.sis.pitt.edu/lti/launch?tool=dbqa' in url
    is_sqlknot = 'http://adapt2.sis.pitt.edu/sqlknot/index.html' in url
    is_turkish = '&lang=TR' in url
    
    if 'http://columbus.exp.sis.pitt.edu/sql_ae/sqlAE.html' in url:
        # not working, so skip for now
        continue
    
    agg_kcs = get_agg_kcs(item)
    agg_kcs_classifiers = set([each['source'] for each in agg_kcs])
    um2_kcs = get_um2_kcs(item)
    um2_kcs_classifiers = set([each['ConceptDescription'] for each in um2_kcs])

    slc_item = {
        "identity": {
            "id": item['short_name'],
            "title": item['name'],
            "type": "DBQA" if is_dbqa else "SQLKnot" if is_sqlknot else "SQL",
        },
        "tags": [],
        "status": "public",
        "user_email": "moh70@pitt.edu",
        "links": {"demo_url": url+"&usr=adl01&grp=adl01"},
        "attribution": {
            "created_at": item['creation_date'],
            "provider": item['provider_id'],
            "publisher": "",
            "authors": [{"name": item['author_name'],"affiliation": affiliations.get(item['author_name'], "")}]
        },
        "languages": {"content_language": "tr" if is_turkish else "en","programming_languages": ["sql"]},
        "content": {"prompt": item['description'],"source_code": ""},
        "classification": {"topics": [],"difficulty": "","knowledge_components": {
            **{c: { 
                'concepts': list(sorted(set([each['name'] for each in agg_kcs if each['source'] == c]))), 'note': '' } 
                for c in agg_kcs_classifiers 
            },
            **{
                f"{c} ": { 'concepts': list(sorted(set([(
                    f"{each['ParentConcept']['t']}::" if 'ParentConcept' in each else ''
                )+each['ConceptTitle'] for each in um2_kcs if each['ConceptDescription'] == c]))), 'note': '' }
                for c in um2_kcs_classifiers
            }
        }},
        "pedagogy": {"learning_objectives": [],"instructional_role": "","prerequisites": {"topics": [],"concepts": [],"item_ids": []}},
        "interaction": {"interaction_type": ""},
        "delivery": [{"format": "html","url": item['url']}],
        "rights": {"license": "MIT","license_url": "","usage_notes": ""},
        "uses": [{
            "context_id": f"cid={course['id']}",
            "context_url": f"http://adapt2.sis.pitt.edu/um-vis-dev2/index.html?usr=demo&grp=adl&sid=adl&cid={course['id']}",
            "context_name": course['name'],
            "used_at": course['creation_date'],
            "used_by": uses_by.get(course['author_name'], course['author_name']),
        } for course in get_used_in(item)],
    }
    
    response = requests.post(
        f"http://adapt2.sis.pitt.edu/next.course-authoring/api/slc-items-api/",
        json=slc_item,
        headers={'api-token': api_token}
    )
    print('status_code:', response.status_code, ' --> ', slc_item['identity']['title'])

processing 1/274: http://adapt2.sis.pitt.edu/lti/launch?tool=dbqa&sub=inner-join-2
status_code: 404  -->  Inner-join query execution 2
processing 2/274: http://adapt2.sis.pitt.edu/lti/launch?tool=dbqa&sub=inner-join-3
status_code: 404  -->  Inner-join query execution 3
processing 3/274: http://adapt2.sis.pitt.edu/lti/launch?tool=dbqa&sub=inner-join-4
status_code: 404  -->  Inner-join query execution 4
processing 4/274: http://adapt2.sis.pitt.edu/lti/launch?tool=dbqa&sub=left-join-2
status_code: 404  -->  Left-join query execution 2
processing 5/274: http://adapt2.sis.pitt.edu/lti/launch?tool=dbqa&sub=left-join-3
status_code: 404  -->  Left-join query execution 3
processing 6/274: http://adapt2.sis.pitt.edu/lti/launch?tool=dbqa&sub=left-join-4
status_code: 404  -->  Left-join query execution 4
processing 7/274: http://adapt2.sis.pitt.edu/lti/launch?tool=dbqa&sub=left-join-1
status_code: 404  -->  Left-join query execution 1
processing 8/274: http://adapt2.sis.pitt.edu/lti/launch?tool=db

KeyboardInterrupt: 